# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

In [1]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

using SCPToolbox
using PyPlot, Colors, LinearAlgebra

# Import the different possible low-level convex solvers
using COSMO
using Ipopt
using SCS
using ECOS
using OSQP

  Activating project at `~/GitHub/SCPToolbox_tutorial`
   Resolving package versions...
  No Changes to `~/GitHub/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/GitHub/SCPToolbox_tutorial/Manifest.toml`


# Part 4: Rocket-Landing Guidance

In [2]:
pbm = TrajectoryProblem();

## Rocket dynamics

In general, the dynamics of any nonlinear system are given by:

$$
\dot x(t) = f( t, x(t), u(t), p )
$$

* $t\in \mathbb R$: time;
* $x(t)\in \mathbb R^n$: state vector;
* $u(t)\in \mathbb R^m$: input vector (the things that we can decide on);
* $p(t)\in \mathbb R^d$: "parameter" vector (problem specific);

In [3]:
n, m, p = 7, 2, 1
problem_set_dims!(pbm, n, m, p)

## Rocket parameters

In [4]:
g = 9.81 # [m/s^2] Gravitational acceleration

9.81

In [5]:
# Mechanical parameters
rs = 4.5 # [m] Fuselage radius
ls = 50.0 # [m] Fuselage height
m_wet = 120e3
L = 0.4 * ls
# lcp = 0.45 * ls
J = 1 / 12 * m_wet * (6 * rs^2 + ls^2);

In [6]:
# Propulsion parameters
Isp = 330 # [s] Specific impulse
T_min1 = 880e3 # [N] One engine min thrust
T_max1 = 2210e3 # [N] One engine max thrust
T_min = 3 * T_min1
T_max = 3 * T_max1
α = 1 / (Isp * g)
δ_max = deg2rad(10.0);

In [7]:
f(t, x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [vx; vy; ω; -(T/m)*sin(θ+δ); (T/m)*cos(θ+δ)-g; -(L*T/J)*sin(δ); -α*T]*tf
end;

SCP algorithms work by iteratively linearizing nonconvex elements of the problem. This means that we have to provide the algorithm with the Jacobians of $f$.

\begin{align}
A(t, x, u, p) &= \nabla_x f(t, x, u, p), \\
B(t, x, u, p) &= \nabla_u f(t, x, u, p), \\
F(t, x, u, p) &= \nabla_p f(t, x, u, p).
\end{align}

In [8]:
A(t, x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [0 0        0      1 0 0        0;
            0 0        0      0 1 0        0;
            0 0        0      0 0 1        0;
            0 0 -T*cos(θ+δ)/m 0 0 0  T*sin(θ+δ)/m^2;
            0 0 -T*sin(θ+δ)/m 0 0 0 -T*cos(θ+δ)/m^2;
            0 0        0      0 0 0        0;
            0 0        0      0 0 0        0        ]*tf
end

B(t, x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [     0            0;
                 0            0;
                 0            0;
            -sin(θ+δ)/m -T*cos(θ+δ)/m;
             cos(θ+δ)/m -T*sin(θ+δ)/m;
            -L*sin(δ)/J -L*T*cos(δ)/J;
                -α            0       ]*tf
end

F(t, x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    return [vx; vy; ω; -(T/m)*sin(θ+δ); (T/m)*cos(θ+δ)-g; -(L*T/J)*sin(δ); -α*T]
end;

In [9]:
wrap(func) = (t, k, x, u, p, pbm) -> func(t, x, u, p)
problem_set_dynamics!(pbm, wrap(f), wrap(A), wrap(B), wrap(F))

## Boundary Conditions

The initial and terminal boundary conditions on the trajectory are set by the following two constraints:

\begin{align}
g_{ic}(x(0), p) &= 0, \\
g_{tc}(x(1), p) &= 0.
\end{align}

In [10]:
x_0 = [-100; 100; 10; -1; deg2rad(60); 0.0; m_wet]
x_f = zeros(6)

g_ic(x, p) = x-x_0
g_tc(x, p) = x[1:6]-x_f;

Again, we need to provide the Jacobians of $g_{ic}$ and $g_{tc}$, since in general these may be nonaffine functions.

\begin{align}
H_0(x(0), p) &= \nabla_x g_{ic}(x(0), p), \\
K_0(x(0), p) &= \nabla_p g_{ic}(x(0), p), \\
H_f(x(1), p) &= \nabla_x g_{tc}(x(1), p), \\
K_f(x(1), p) &= \nabla_p g_{tc}(x(1), p).
\end{align}

When a Jacobian is not provided, the SCP Toolbox assumes that it is zero.

In [11]:
H_0(x, p) = I(7)
H_f(x, p) = hcat(I(6), zeros(6));

In [12]:
wrap(func) = (x, p, pbm) -> func(x, p)
problem_set_bc!(pbm, :ic, wrap(g_ic), wrap(H_0))
problem_set_bc!(pbm, :tc, wrap(g_tc), wrap(H_f))

## State constraints

In [13]:
# tf_min = 10
# tf_max = 1000

# time_upper(t, x, p) = begin
#     tf, = p
#     return [tf - tf_max]
# end

# time_lower(t, x, p) = begin
#     tf, = p
#     return [tf_min - tf]
# end

# wrap(func) = (ocp, t, k, x, p) -> func(t, x, p)
# problem_set_X!(pbm, wrap(time_upper))
# problem_set_X!(pbm, wrap(time_lower))

# # Convex path constraints on the state
# tf_min = 20
# tf_max = 100
# problem_set_X!(
#     pbm, (t, k, x, p, pbm, ocp) -> begin
        
#         tf, = p
        
#         @add_constraint(
#             ocp, NONPOS, "max_time", (tf,), begin
#                 local tf = arg
#                 tf-tf_max
#             end)

#         @add_constraint(
#             ocp, NONPOS, "min_time", (tf,), begin
#                 local tf = arg
#                 tf_min-tf
#             end)
#     end)

## Control constraints

In [14]:
# thrust_upper(t, u, p) = begin
#     T, δ = u
#     return [T - T_max]
# end

# thrust_lower(t, u, p) = begin
#     T, δ = u
#     return [T_min - T]
# end

# gimbal_upper(t, u, p) = begin
#     T, δ = u
#     return [δ - δ_max]
# end

# wrap(func) = (t, k, u, p, pbm, ocp) -> func(t, u, p)
# problem_set_U!(pbm, wrap(thrust_upper))
# problem_set_U!(pbm, wrap(thrust_lower))
# problem_set_U!(pbm, wrap(gimbal_upper))

# # Convex path constraints on the input
# problem_set_U!(
#     pbm, (t, k, u, p, pbm, ocp) -> begin
        
#         T, δ = u

#         @add_constraint(
#             ocp, NONPOS, "max_thrust", (T,), begin
#                 local T = arg[1]
#                 T[1]-T_max
#             end)

#         @add_constraint(
#             ocp, NONPOS, "min_thrust", (T,), begin
#                 local T = arg[1]
#                 T_min-T[1]
#             end)

#         @add_constraint(
#             ocp, L1, "gimbal", (δ,), begin
#                 local δ = arg[1]
#                 vcat(δ_max, δ)
#             end)
#     end)

## Objective Function

Borrowing from the standard choice in optimal control literature, the SCP Toolbox works with an objective function in the Bolza form:

$$
J(x, u, p) = \phi(x(1), p) + \int_0^1 \Gamma(x(t), u(t), p) dt.
$$

The function $\phi(\cdot)\in\mathbb R$ defines the terminal cost, while $\Gamma(\cdot)\in\mathbb R$ defines the running cost. Both have to be convex, at most quadratic, functions.

> ℹ️ Any nonconvexity in the cost can in general be off-loaded into the constraints by defining additional state, control, and/or parameter variables.

In [15]:
ϕ(x, p) = -x[end];

The SCP Toolbox provides the following API functions for defining the cost:
* `problem_set_terminal_cost!`: if not provided, it is assumed that $\phi\equiv 0$;
* `problem_set_running_cost!`: if not provided, it is assumed that $\Gamma\equiv 0$;

In [16]:
wrap(func) = (x, p, pbm) -> func(x, p)
problem_set_terminal_cost!(pbm, wrap(ϕ))

## Initial Trajectory Guess

SCP algorithms require an initial guess for the trajectory. This can be very simple, and does not need to be feasible.

For Dubin's car, we will use a straight line for the state and zero for the input.

Given a time $t\in [0,1]$, the state initial guess will be:

\begin{align}
\bar x(t) &= (1-t) x_0 + t x_f, \\
\bar u(t) &= 0.
\end{align}

SCP algorithms work by discretizing the trajectory problem into $N$ temporal nodes.

The API provides a function `problem_set_guess!` for defining the initial guess, discretized into $N$ points.

For convenience, a function `straightline_interpolate` is available to define a straight line guess.

In [17]:
state_guess(N) = straightline_interpolate(x_0, vcat(x_f, 100e3), N)
input_guess(N) = straightline_interpolate([T_min; 0.0], [T_min; 0.0], N);

In [18]:
problem_set_guess!(pbm, (N, pbm) -> begin
    x = state_guess(N)
    u = input_guess(N)
    p = [60.0] # zeros(1)
    return x, u, p
end)

## Configuring the SCP Solver

The SCP Toolbox provides several SCP algorithm choices: `PTR`, `SCvx`, and `GuSTO`.

Each algorithm must be provided with a set of parameters that define its behavior. Most of the parameters are shared, but some are algorithm-specific.

SCP algorithm performance depends significantly on a well-chosen set of parameters, and this is problem-dependent.

A lot of intuition is documented in our [Control Systems Magazine article](https://arxiv.org/abs/2106.09125).

In [19]:
# Parameters
N, Nsub = 21, 20
iter_max = 30
disc_method = FOH
wvc, wtr = 1e3, 1e0
feas_tol = 5e-3
ε_abs, ε_rel = 1e-5, 1e-3
q_tr = Inf
q_exit = Inf
solver, solver_options = ECOS, Dict("verbose"=>1)      

pars = PTR.Parameters(N, Nsub, iter_max, disc_method, wvc, wtr, ε_abs,
                      ε_rel, feas_tol, q_tr, q_exit, solver, solver_options);

Alternative convex solver options:

In [20]:
# solver, options = Gurobi, Dict("OutputFlag"=>0)  # works
# solver, options = OSQP, Dict("verbose"=>0)       # works
# solver, options = Mosek, Dict("MSK_IPAR_LOG"=>0)
# solver, options = Ipopt, Dict()                  # doesn't work
# solver, options = COSMO, Dict()                  # doesn't work
# solver, options = SCS, Dict()                    # doesn't work

## Solving the Trajectory Problem

Solving the problem amounts to initializing the SCP solver with its parameters, and providing the problem definition `pbm` from the API.

In [21]:
ptr_pbm = PTR.create(pars, pbm)
sol, history = PTR.solve(ptr_pbm);


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +0e+00  0e+00  1e+00  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  -8.918e+01  -0.000e+00  +0e+00  0e+00  1e+00  9e+01  1e-02  0.9890  1e-04   0  0  0 |  0  0

UNBOUNDED (within feastol=0.0e+00).
Runtime: 0.000104 seconds.


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +0e+00  0e+00  1e+00  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  -8.918e+01  -0.000e+00  +0e+00  0e+00  1e+00  9e+01  1e-02  0.9890  1e-04   0  0  0 |  0  0

UNBOUNDED (within feastol=0.0e+00).
Runtime: 0.000026 seconds.


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost

LoadError: MethodError: no method matching *(::Vector{Float64}, ::Vector{Float64})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/matmul.jl:44
[0m  *(::StridedVecOrMat, [91m::Adjoint{<:Any, <:LinearAlgebra.LQPackedQ}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/lq.jl:266
[0m  ...

## Trajectory plot

In [22]:
# function set_fonts()::Nothing
#     # Set the figure fonts.
#     fig_small_sz = 12
#     fig_med_sz = 15
#     fig_big_sz = 17
#     plt.rc("text", usetex=true)
#     plt.rc("font", size=fig_small_sz, family="serif")
#     plt.rc("axes", titlesize=fig_small_sz)
#     plt.rc("axes", labelsize=fig_med_sz)
#     plt.rc("xtick", labelsize=fig_small_sz)
#     plt.rc("ytick", labelsize=fig_small_sz)
#     plt.rc("legend", fontsize=fig_small_sz)
#     plt.rc("figure", titlesize=fig_big_sz)
#     plt.rc("figure", dpi=300) 
#     return nothing
# end
# ;

In [23]:
# # Trajectory plot
# ctres, overlap = 1000, 3
# N = size(sol.xd, 2)
# xct = hcat([sample(sol.xc, t) for t in LinRange(0, 1, ctres)]...)
# vct = vcat([sample(sol.uc, t)[1] for t in LinRange(0, 1, ctres)]...)
# cmap = generate_colormap("inferno"; minval=minimum(vct), maxval=maximum(vct))

# # plot_options = Dict("xtick.labelsize"=>9,
# #                     "ytick.labelsize"=>9,
# #                     "axes.labelsize"=>11)
# # fig = create_figure((4, 4); options = plot_options)

# fig = plt.figure(figsize=(4,4), dpi=300)
# plt.clf()
# set_fonts()
# set_fonts()

# ax = setup_axis!(111, xlabel="\$x\$ [m]", ylabel="\$y\$ [m]",
#                  axis="equal", cbar=cmap, clabel="Velocity, \$v\$ [m/s]",
#                  cbar_aspect=40)
# ax.plot(sol.xd[1, :], sol.xd[2, :],
#         linestyle="none", marker="o", markerfacecolor=DarkBlue,
#         markeredgecolor="white", markeredgewidth=0.2, markersize=3,
#         zorder=20)
# line_segs = Vector{Matrix}(undef, 0)
# line_clrs = Vector{NTuple{4, Real}}(undef, 0)
# for k=1:ctres-overlap
#     push!(line_segs, xct[1:2, k:k+overlap]')
#     push!(line_clrs, cmap.to_rgba(vct[k]))
# end
# trajectory = PyPlot.matplotlib.collections.LineCollection(
#     line_segs, zorder=10, colors = line_clrs, linewidths=3,
#     capstyle="round")
# ax.add_collection(trajectory)
# Rect = PyPlot.matplotlib.patches.Rectangle
# car_length = 0.2
# for k=1:N
#     local xl, xw = [1;1;-1;-1;1]*car_length/2, [1;-1;-1;1;1]*car_width/2
#     local yl, yw = [1;1;-1;-1;1]*car_length/2, [-1;1;1;-1;-1]*car_width/2
#     local ang = sol.xd[3,k]
#     local xc = sol.xd[1,k].+xl.*sin(ang).+xw.*cos(ang)
#     local yc = sol.xd[2,k].+yl.*cos(ang).+yw.*sin(ang)
#     ax.fill(xc, yc,
#             linewidth=1,
#             edgecolor=DarkBlue,
#             facecolor=rgb2pyplot(parse(RGB, Red), a=0.5),
#             zorder=6)
# end
# ang = LinRange(0, 2*pi, 100)
# obs = ([cos.(ang)'; sin.(ang)']*r_0).+c_0
# ax.fill(obs[1, :], obs[2, :],
#         linewidth=1,
#         edgecolor=Blue,
#         facecolor=rgb2pyplot(parse(RGB, Green), a=0.5),
#         zorder=5)

# fig.savefig("../media/outputs/dubin_trajectory.png", dpi="figure", bbox_inches=nothing)

# plt.close()

## Replace figure

<!-- <div style="text-align: center">
    <img src="../media/outputs/dubin_trajectory.png"
         alt="Dubin's car overview"
         style="background-color: white; width: 500px; display: block; margin-left: auto; margin-right: auto;"/>
    <br />
    <b>Figure.</b> Computed Dubin's car trajectory for minimum-input usage.
</div> -->